In [ ]:
!pip install scikit-surprise

In [ ]:
import numpy as np
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [ ]:
data=Dataset.load_builtin('ml-100k', prompt=False)
data.raw_ratings[:10]

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [ ]:
model = SVD()

In [ ]:
cross_validate(model, data, measures=['rmse', 'mae'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9378  0.9394  0.9386  0.9403  0.9326  0.9378  0.0027  
MAE (testset)     0.7381  0.7418  0.7395  0.7427  0.7336  0.7391  0.0032  
Fit time          3.85    3.88    3.83    3.80    3.78    3.83    0.03    
Test time         0.12    0.19    0.14    0.20    0.13    0.16    0.03    


{'fit_time': (3.846891164779663,
  3.878479480743408,
  3.8257880210876465,
  3.8048315048217773,
  3.776750087738037),
 'test_mae': array([0.73808293, 0.74181335, 0.73953768, 0.74269106, 0.73355721]),
 'test_rmse': array([0.93784811, 0.939443  , 0.93863114, 0.94029935, 0.93264951]),
 'test_time': (0.12445807456970215,
  0.18890738487243652,
  0.13811135292053223,
  0.19702482223510742,
  0.13346099853515625)}

#### 컨텐츠 기반 필터링(Content-based Filtering)
 - 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
  - ex) 내가 지금까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
   - 유사도를 기반으로 추천
   - 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
      - 장점
        - 많은 수의 사용자를 대상으로 쉽게 확장 가능
        - 사용자가 관심을 갖지 않던 상품 추천 가능
      - 단점
        - 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
        - 사용자의 기존 관심사항을 기반으로만 추천 가능

In [ ]:
import numpy as np
from surprise import Dataset

- 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기
- 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)
raw_data=np.array(data.raw_ratings, dtype=int)

In [ ]:
raw_data[:,0], len(raw_data[:,0])

(array([196, 186,  22, ..., 276,  13,  12]), 100000)

In [ ]:
# 0부터 시작하기 위해 1를 뺌????
raw_data[:,0] -= 1
raw_data[:,1] -= 1

In [ ]:
raw_data[:,0], len(raw_data[:,0])

(array([195, 185,  21, ..., 275,  12,  11]), 100000)

In [ ]:
n_users=np.max(raw_data[:, 0]) # 총 유저수
n_movies=np.max(raw_data[:, 1]) #  영화의 총 갯수
shape = (n_users+1, n_movies+1) # 전체 매트릭스 크기 확인
shape

(943, 1683)

In [ ]:
adj_matrix=np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, timie in raw_data:
  adj_matrix[user_id][movie_id] = 1.
adj_matrix

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    similarity = np.dot(my_vector, user_vector)
    if similarity > best_match:
      best_match = similarity
      best_match_id = user_id
      best_match_vector user_vector

print('Best Match : {}, Best Match ID: {}'.format(best_match, best_match_id))

In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 >0.:
    recommend_list.append(i)
print(recommend_list)

- 유클리드 거리를 사용해 추천
- 거리가 가까울 수록(값이 작을수록) 나와 유사한 사용자

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector-user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector user_vector

print('Best Match : {}, Best Match ID: {}'.format(best_match, best_match_id))

In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 >0.:
    recommend_list.append(i)
print(recommend_list)

- 코사인 유사도를 사용해 추천

- 두 벡터가 이루고 있는 각을 계산

In [ ]:
def compute_cos_similarity(v1, v2):
  norm1 = np.sqrt(np.sum(np.square(v1)))
  norm2 = np.sqrt(np.sum(np.square(v2)))
  dot = np.dot(v1,v2)
  return dot / (norm1* norm2)

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if euclidean_dist < best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector user_vector

print('Best Match : {}, Best Match ID: {}'.format(best_match, best_match_id))

기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [ ]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating
  adj_matrix  

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector-user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector user_vector

print('Best Match : {}, Best Match ID: {}'.format(best_match, best_match_id))

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id !=user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if euclidean_dist < best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector user_vector

print('Best Match : {}, Best Match ID: {}'.format(best_match, best_match_id))

협업 필터링(Collaborative Filtering)
  - 사용자와 항목의 유사성을 동시에 고려해 추천
  - 기존에 내 관심사가 아닌 항목이라도 추천 가능
  - 자동으로 임베딩 학습 가능
  - 협업 필터링은 다음과 같은 장단점을 갖고 있다.

    - 장점
      - 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요없다.
      - 기존의 관심사가 아니더라도 추천 가능
    - 단점
      - 학습 과정에 나오지 않는 항목은 임베딩을 만들 수 없음
      - 추가 특성을 사용하기 어려움